In [45]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import requests
import pathlib
import json
from datetime import datetime
from meteostat import Stations, Daily
import matplotlib.pyplot as plt


In [46]:
# Load API keys
from dotenv import load_dotenv
import os
load_dotenv() 
print((os.getenv("VISUAL_CROSSING_API_KEY")))

MHFU2QHX7NTY5RTWZPAT7VBXS


In [56]:
# Define some constants
latitude = [40.79736, 41.78701, 30.1444, 25.7738]
longitude = [-73.97785, -87.77166, -97.66876, -80.1936]
cities = ["ny", "il", "tx", "fl"]
stations_ncei=["USW00094728", "USW00014819", "USW00013904", "USC00086315"]
start_date = "2016-01-01"
end_date = "2024-03-24"

## API Calls to collect historical weather data

### Open Meteo

In [48]:
def getDataFromOpenMeteo(latitude, longitude, startDate, endDate, fileName):
  # Data Source 1
	# Setup the Open-Meteo API client with cache and retry on error
	cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
	retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
	openmeteo = openmeteo_requests.Client(session = retry_session)

	# Make sure all required weather variables are listed here
	# The order of variables in hourly or daily is important to assign them correctly below
	url = "https://archive-api.open-meteo.com/v1/archive"
	params = {
		"latitude": latitude,
		"longitude": longitude,
		"start_date": startDate,
		"end_date": endDate,
		"daily": ["temperature_2m_max", "temperature_2m_min", "sunshine_duration", "precipitation_hours", "wind_speed_10m_max"],
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_temperature_2m_max = daily.Variables(0).ValuesAsNumpy()
	daily_temperature_2m_min = daily.Variables(1).ValuesAsNumpy()
	daily_sunshine_duration = daily.Variables(2).ValuesAsNumpy()
	daily_precipitation_hours = daily.Variables(3).ValuesAsNumpy()
	daily_wind_speed_10m_max = daily.Variables(4).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}
	daily_data["temperature_2m_max"] = daily_temperature_2m_max
	daily_data["temperature_2m_min"] = daily_temperature_2m_min
	daily_data["sunshine_duration"] = daily_sunshine_duration
	daily_data["precipitation_hours"] = daily_precipitation_hours
	daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max

	daily_dataframe = pd.DataFrame(data = daily_data)
	daily_dataframe.to_csv("openMeteo_" + '_'.join([fileName, startDate, 'to', endDate]) +  ".csv", index=False)
	return daily_dataframe
	# print(daily_dataframe)


In [51]:
# WARNING!!!
# This has already been run for 2016-01-01 to 2024-03-12
# Don't re-run and make repeated API calls unless needed
# Get data from OpenMeteo

daily_data = []
for i in range(len(latitude)):
  daily_data.append(getDataFromOpenMeteo(latitude[i], longitude[i], start_date, end_date, cities[i]))

print(len(daily_data))

Coordinates 40.808433532714844°N -74.0198974609375°E
Elevation 0.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 41.79261779785156°N -87.78262329101562°E
Elevation 187.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 30.123022079467773°N -97.67523193359375°E
Elevation 154.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 25.764497756958008°N -80.19607543945312°E
Elevation 7.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
4


### Visual Crossing

In [40]:
def getDataFromVisualCrossing(latitude, longitude, startDate, endDate, fileName):
  url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/" + str (latitude) + \
        "%2C" + str(longitude) + "/" + startDate + "/" + endDate + "?unitGroup=us&include=days&key="+ os.getenv("VISUAL_CROSSING_API_KEY") + "&contentType=json"
  print(url)
  print("https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/40.79736%2C-73.97785/2016-01-01/today?unitGroup=us&include=days&key=MHFU2QHX7NTY5RTWZPAT7VBXS&contentType=json")
# "https://weather.visualcrossing.com/VisualCrosingWebServices/rest/services/timeline/
# 40.79736%2C-73.97785/2016-01-01/today?unitGroup=us&include=days&key=MHFU2QHX7NTY5RTWZPAT7VBXS&contentType=json"


  payload={}
  headers = {}

  response = requests.request("GET", url, headers=headers, data=payload)
  pathlib.Path("visualCrossing_" + '_'.join([fileName, startDate, 'to', endDate]) + '.json').write_bytes(response.content)

  print(response.text)

In [41]:
# WARNING!!!
# Will incur an API cost, don't re-run
# Historical Data is saved to a CSV

# visual_crossing_data = []
# for i in range(len(latitude)):
#   visual_crossing_data.append(getDataFromVisualCrossing(latitude[i], longitude[i], start_date, end_date, cities[i]))

https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/40.79736%2C-73.97785/2024-03-20/2024-03-22?unitGroup=us&include=days&key=MHFU2QHX7NTY5RTWZPAT7VBXS&contentType=json
https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/40.79736%2C-73.97785/2016-01-01/today?unitGroup=us&include=days&key=MHFU2QHX7NTY5RTWZPAT7VBXS&contentType=json
{"queryCost":3,"latitude":40.79736,"longitude":-73.97785,"resolvedAddress":"40.79736,-73.97785","address":"40.79736,-73.97785","timezone":"America/New_York","tzoffset":-4.0,"days":[{"datetime":"2024-03-20","datetimeEpoch":1710907200,"tempmax":55.6,"tempmin":37.1,"temp":45.0,"feelslikemax":55.6,"feelslikemin":28.0,"feelslike":41.8,"dew":27.6,"humidity":51.5,"precip":0.013,"precipprob":100.0,"precipcover":4.17,"preciptype":["rain"],"snow":0.0,"snowdepth":0.0,"windgust":37.0,"windspeed":17.1,"winddir":272.9,"pressure":1004.9,"cloudcover":82.7,"visibility":9.9,"solarradiation":76.3,"solarenergy":6.5,"u

### Meteostat

In [42]:
# Get weather stations
stations = Stations()
required_stations = []
for i in range(len(latitude)):
  stations = stations.nearby(latitude[i], longitude[i])
  station = stations.fetch(1)
  required_stations.append(station)

# print(required_stations)

In [43]:
def getDataFromMeteostat(latitude, longitude, startDate, endDate, fileName):
  start = datetime.strptime(startDate, '%Y-%m-%d')
  end = datetime.strptime(endDate, '%Y-%m-%d')
  stations = Stations()
  stations = stations.nearby(latitude, longitude)
  station = stations.fetch(1)
  # Get daily data
  data = Daily(station, start, end)
  data = data.fetch()
  # print(data['time'])
  data.index.names = ['date']
  data = data.add_suffix('_ms')
  data.to_csv("meteoStat_" + '_'.join([fileName, startDate, 'to', endDate]) +  ".csv")
  return data
  # data.plot(y=['tavg', 'tmin', 'tmax'])
  # plt.show()


In [44]:
daily_ms_data = []
for i in range(len(latitude)):
  daily_ms_data.append(getDataFromMeteostat(latitude[i], longitude[i], start_date, end_date, cities[i]))


### NCEI

In [54]:

# Already downloaded manually as CSV
def getDataFromNCEI(station, startDate, endDate, fileName):
    base_url = "https://www.ncei.noaa.gov/access/services/data/v1?"
    # Define the query parameters
    params = {
        "dataset": "daily-summaries",  
        "stations": station,
        "startDate": start_date, 
        "endDate": end_date,
        "format": "json",
        "units": "standard"
    }

    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
    else:
        print("Error. Status code:", response.status_code)

    # Convert the json data from the NOAA to a dataframe
    data_df = pd.DataFrame.from_records(data)
    data_df.to_csv("ncei_" + '_'.join([fileName, startDate, 'to', endDate]) +  ".csv", index=False)


In [57]:
for i in range(len(stations_ncei)):
  getDataFromNCEI(stations_ncei[i], start_date, end_date, cities[i])

## Reading Data from the CSV and JSON Files created from the API calls

In [6]:
def readStoredJSONData(fileName):
  with open(fileName, 'r') as file:
    # Reading from json file
    data = json.load(file)
  return data

def readStoredCSVData(fileName):
  df = pd.read_csv(fileName)
  return df

In [7]:
# Read all visual crossing files
vc_data = []
for i in range(len(latitude)):
  fileName = "visualCrossing_" + '_'.join([cities[i], start_date, 'to', end_date]) + '.json'
  vc_data.append(readStoredJSONData(fileName))

In [10]:
type(vc_data[0]['days'])


list

In [11]:
for cityData in vc_data:
  city_df = pd.DataFrame(cityData['days'])
  city_df = city_df[['datetime', 'tempmax', 'tempmin', 'humidity', 'windspeed']]
  city_df['datetime'] = city_df['datetime'].apply(lambda x : str(x))
  print(city_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   3002 non-null   object 
 1   tempmax    3002 non-null   float64
 2   tempmin    3002 non-null   float64
 3   humidity   3002 non-null   float64
 4   windspeed  3002 non-null   float64
dtypes: float64(4), object(1)
memory usage: 117.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   3002 non-null   object 
 1   tempmax    3002 non-null   float64
 2   tempmin    3002 non-null   float64
 3   humidity   3002 non-null   float64
 4   windspeed  3002 non-null   float64
dtypes: float64(4), object(1)
memory usage: 117.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 5 columns):
 #  

In [12]:
vc_dfs = []
for cityData in vc_data:
  city_df = pd.DataFrame(cityData['days'])
  city_df = city_df[['datetime', 'tempmax', 'tempmin', 'humidity', 'windspeed']]
  print(city_df.info())
  vc_dfs.append(city_df)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   3002 non-null   object 
 1   tempmax    3002 non-null   float64
 2   tempmin    3002 non-null   float64
 3   humidity   3002 non-null   float64
 4   windspeed  3002 non-null   float64
dtypes: float64(4), object(1)
memory usage: 117.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   datetime   3002 non-null   object 
 1   tempmax    3002 non-null   float64
 2   tempmin    3002 non-null   float64
 3   humidity   3002 non-null   float64
 4   windspeed  3002 non-null   float64
dtypes: float64(4), object(1)
memory usage: 117.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002 entries, 0 to 3001
Data columns (total 5 columns):
 #  

In [13]:
# Read all open meteo files
om_dfs = []
for i in range(len(latitude)):
  fileName = "openMeteo_" + '_'.join([cities[i], start_date, 'to', end_date]) + '.csv'
  om_df = readStoredCSVData(fileName)
  # print(type(om_df['date'][0]))
  # om_df['date'] = om_df['date'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
  om_df['date'] = om_df['date'].apply(lambda x: x[:10])
  om_df.set_index('date')
  om_dfs.append(om_df)

In [15]:
om_dfs[0].tail()

date  temperature_2m_max  temperature_2m_min  sunshine_duration  \
2997  2024-03-16           14.292999               4.843          38986.780   
2998  2024-03-17           15.493000               7.393          36546.020   
2999  2024-03-18            9.943000               4.093          34802.510   
3000  2024-03-19            7.343000               0.593          30800.836   
3001  2024-03-20                 NaN                 NaN                NaN   

      precipitation_hours  wind_speed_10m_max  
2997                  0.0           18.486622  
2998                  2.0           28.440000  
2999                  0.0           19.995398  
3000                  0.0           26.980793  
3001                  0.0                 NaN

In [16]:
ms_dfs = []
for i in range(len(latitude)):
  fileName = "meteoStat_" + '_'.join([cities[i], start_date, 'to', end_date]) + '.csv'
  ms_df = readStoredCSVData(fileName)
  ms_df['date'] = ms_df['date'].apply(lambda x: x[:10])
  ms_df.set_index('date')
  ms_dfs.append(ms_df)

In [17]:
ncei_dfs = []
for i in range(len(latitude)):
  fileName = "ncei_" + '_'.join([cities[i]]) + '.csv'
  # print(fileName)
  ncei_df = readStoredCSVData(fileName)
  ncei_df.columns = map(str.lower, ncei_df.columns)
  # ncei_df['date'] = ncei_df['date'].apply(lambda x: x[:10])
  ncei_df = ncei_df.add_suffix('_ncei')
  ncei_df = ncei_df.rename(columns={'date_ncei': 'date'})
  ncei_df.set_index('date')
  ncei_dfs.append(ncei_df)
  # print(ncei_df.info())

In [24]:
for vc_df in vc_dfs:
  vc_df.columns = ['date', 'tmax_vc', 'tmin_vc', 'humi_vc', 'wind_vc']
  vc_df.set_index('date')
  
for om_df in om_dfs:
  om_df.columns = ['date', 'tmax_om', 'tmin_om', 'sund_om', 'prec_om', 'wind_om']
  om_df.set_index('date')

for ms_df in ms_dfs:
  ms_df.set_index('date')
  
for ncei_df in ncei_dfs:
  # ncei_df = ncei_df.rename(columns={'date_ncei': 'date'})
  ncei_df.set_index('date')
  # print(ncei_df.info())

# print(vc_dfs[0].head())
# print(type(vc_dfs[0]['date']))
# print(om_dfs[0].head())
# print(type(om_dfs[0]['date']))
# print(ms_dfs[0].info())
# print(ncei_dfs[0].info())


merged_dfs = []

for i in range(len(vc_dfs)):
  merged_df = pd.merge(vc_dfs[i], om_dfs[i])
  merged_df = pd.merge(merged_df, ms_dfs[i]) 
  # merged_df = pd.merge(merged_df, ncei_dfs[i]) 
  # , left_index=True, right_index=True)
  merged_dfs.append(merged_df)

print(merged_dfs[0].info())
print(merged_dfs[0].tail())

# Merged_dfs is a single DF will all data points from different sources

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458 entries, 0 to 1457
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   date     1458 non-null   object 
 1   tmax_vc  1458 non-null   float64
 2   tmin_vc  1458 non-null   float64
 3   humi_vc  1458 non-null   float64
 4   wind_vc  1458 non-null   float64
 5   tmax_om  1457 non-null   float64
 6   tmin_om  1457 non-null   float64
 7   sund_om  1457 non-null   float64
 8   prec_om  1458 non-null   float64
 9   wind_om  1457 non-null   float64
 10  tavg_ms  1458 non-null   float64
 11  tmin_ms  1458 non-null   float64
 12  tmax_ms  1458 non-null   float64
 13  prcp_ms  1170 non-null   float64
 14  snow_ms  0 non-null      float64
 15  wdir_ms  1170 non-null   float64
 16  wspd_ms  1170 non-null   float64
 17  wpgt_ms  0 non-null      float64
 18  pres_ms  1170 non-null   float64
 19  tsun_ms  0 non-null      float64
dtypes: float64(19), object(1)
memory usage: 227.9+ KB
No

## Serialising and De-serialising the final DFs for the historical data

In [27]:
print(merged_dfs[1]['date'].tail())

2997    2024-03-16
2998    2024-03-17
2999    2024-03-18
3000    2024-03-19
3001    2024-03-20
Name: date, dtype: object


In [32]:
# Store the merged_df DataFrame to disk for later computations
for i in range(len(cities)):
  merged_dfs[i].to_pickle("./merged_df_" + cities[i] + ".pkl")  

In [31]:
# Unpickle the DataFrames
city_history_dfs = []

for i in range(len(cities)):
  city_history_dfs.append(pd.read_pickle("./merged_df_" + cities[i] + ".pkl"))

print("Loaded DFs for " + str(len(city_history_dfs)) + " cities.\n")
# print(city_history_dfs[0].info())
print(city_history_dfs[0].tail())

Loaded DFs for 4 cities.

            date  tmax_vc  tmin_vc  humi_vc  wind_vc    tmax_om  tmin_om  \
1453  2024-03-16     60.5     46.9     52.9     10.1  14.292999    4.843   
1454  2024-03-17     61.8     51.0     43.6     19.6  15.493000    7.393   
1455  2024-03-18     49.4     39.8     37.2     15.3   9.943000    4.093   
1456  2024-03-19     47.3     35.9     42.7     14.6   7.343000    0.593   
1457  2024-03-20     55.6     37.1     51.5     17.1        NaN      NaN   

        sund_om  prec_om    wind_om  tavg_ms  tmin_ms  tmax_ms  prcp_ms  \
1453  38986.780      0.0  18.486622     11.4      6.3     16.1      0.0   
1454  36546.020      2.0  28.440000     12.9      9.1     16.7      0.2   
1455  34802.510      0.0  19.995398      7.3      4.0      9.4      0.0   
1456  30800.836      0.0  26.980793      4.1      2.0      7.5      0.2   
1457        NaN      0.0        NaN      6.1      1.5     12.0      0.6   

      snow_ms  wdir_ms  wspd_ms  wpgt_ms  pres_ms  tsun_ms  
1453 